In [1]:
from osgeo import gdal

In [2]:
import os
from utils.GenUtils import make_folder, get_paths, chunk_creator, folder_file_size, question

In [3]:
os.environ["ISISROOT"]="/opt/conda/envs/isis/"
os.environ["ISISDATA"]="/isis/data"

In [4]:
import kalasiris as isis

In [5]:
config = {
            "inst":"lronac",
            "maptemplate":"../PyISIS-Parallel/maptemplates/DomeEquirectangular.map",
            "oxt":"tiff",
            "PATH":"/home/jovyan/data/",
            "DSTPATH":"/home/jovyan/data/"
            }

In [7]:
if config['inst']=='lronac':
    from utils.KalaUtils import lro
    proc_func=lro
    cam = 'nac'
    ixt='IMG'
elif config['inst']=='lrowac':
    from utils.KalaUtils import lro
    proc_func=lro
    cam = 'wac'
    ixt='IMG'
elif config['inst']=='m3':
    from utils.KalaUtils import m3
    proc_func=m3
    ixt='LBL'
else:
    print('Cannot find utils')
    
maptemplate=config['maptemplate']

In [8]:
def mapper(src, proc_func, maptemplate, ixt, oxt, cam):
    src_basename = os.path.basename(src).split('.'+ixt)[0]
    dst_basename = DSTPATH+'/'+src_basename
    print(dst_basename)
#    if os.path.isfile(dst_basename+'_lev2.JP2'):
#        print ("File exist")
#    elif os.path.isfile(dst_basename+'.JP2'):
#        print ("File exist")
#    else:
    try:
        proc_func(src, dst_basename, maptemplate, oxt, cam)
    except Exception as e:
        print(e)
        
  

In [9]:
def parallel_mapper(files, JOBS, proc_func, maptemplate, ixt, oxt, cam):
    from joblib import Parallel, delayed, parallel_backend
    with parallel_backend("loky", inner_max_num_threads=2):

        Parallel (n_jobs=JOBS)(delayed(mapper)(files[i],proc_func, maptemplate, ixt, oxt, cam)
                                for i in range(len(files)))

In [10]:
def main(PATH, DPATH, ixt, maptemplate):
    image_list = get_paths(PATH, ixt) 
    total_size, max_size, av_fsize = folder_file_size(PATH,image_list)
    from tqdm import tqdm
    import psutil
    avram=psutil.virtual_memory().total >> 30
    avcores=psutil.cpu_count(logical=False)
    avthreads=psutil.cpu_count(logical=True)
    ram_thread = avram/avthreads
    req_mem = avthreads*max_size
    if req_mem > avcores and req_mem > avram:
        JOBS = int(avcores - avcores/(avram/avcores))
    else:
        JOBS = int(avcores - avcores/(avram/avcores))
    #JOBS = 
    with tqdm(total=len(image_list),
             desc = 'Generating Images',
             unit='File') as pbar:
        filerange = len(image_list)
        chunksize = round(filerange/JOBS)
        if chunksize <1:
            chunksize=1
            JOBS = filerange
        chunks = []
        for c in chunk_creator(image_list, JOBS):
            chunks.append(c)
        # from time import time
        from datetime import datetime
        for i in range(len(chunks)):
            start = datetime.now()
            dt_string = start.strftime("%d/%m/%Y %H:%M:%S")
            #print(f'Loop {i} started at: {dt_string}', chunks[i])
            files = chunks[i]
            parallel_mapper(files, JOBS, proc_func, maptemplate, ixt, config['oxt'], cam)
            pbar.update(JOBS)

In [11]:
if __name__ == "__main__":
    PATH=config['PATH']
    DSTPATH=config['DSTPATH']
    os.makedirs(DSTPATH, exist_ok=True)
    #print(ixt)
    main(PATH, DSTPATH, ixt, maptemplate)

Generating Images:   0%|                                                                                                                                                         | 0/1 [00:00<?, ?File/s]

/home/jovyan/data/test/processed//m1335299652le
Spiceinit
Done


Generating Images: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [09:51<00:00, 591.01s/File]


In [12]:
image_list = get_paths(DSTPATH, 'tiff') 
len(image_list)

1

In [13]:
from utils.KalaUtils import equalizer

In [14]:
def equalizer(L2list):
    L2equ = [cub.split('.cub')[0]+'_equ.cub' for cub in L2list]
    isis.equalizer(fromList=L2list, to=L2equ)
    return(L2equ)

In [15]:
L2list = [lev2 for lev2 in get_paths(DSTPATH,'cub') if 'lev2.cub' in lev2]
type(L2list)


list

In [17]:
isis.equalizer(L2list)

CalledProcessError: Command '['equalizer', "from=['/home/jovyan/data/test/processed//m1335299652le_lev2.cub']"]' returned non-zero exit status 4.